# Typical activation potential: analysis of each proof N against context

This notebook implements the *typical activation potential* from `main.tex`.

Notation (from `main.tex`):
- Phi_T(r, C) is the typical activation potential for resource r in context C.
- Phi_h(r, C) is the historical component of activation potential.
- Phi_Tc(r, C) is the typical co-occurrence component (Phi_{T_c}).
- delta (DELTA in code) is the weight in Phi_T = delta * Phi_h + (1 - delta) * Phi_Tc, with delta in [0, 1].

Assumptions (matching the existing SPARQL queries):

Assumption: The SPARQL queries used in this notebook were verified to implement the paper's definitions of
context C and "together" (per definition/postulate/common notion or proposition/proof).
If query scopes are changed, results may no longer match the formulas in `main.tex`.
- Context C for proof n are the resources in definitions, postulates, common notions, propositions up to n (included), and proofs up to n-1 (included)
- "Together" for co-occurrence means resources that co-occur within the same definition, postulate, common notion, proposition, or proof.
- Phi_h is computed from history queries; Phi_Tc is computed from Hebbian pair degrees (co-occurrence links); Phi_T uses the weighted sum above.
- Empty denominators yield 0 for the corresponding potential.
- TYPE_SELECTION toggles type-based co-occurrence for propositions/proofs (relation/operation types) when true.


# SPEC

GOALS: 
(1) Apply the description of typical activation potential provided in main.tex to compute the activation potential of resources used in proof N against the context of resources used in definitions, postulates, common notions, propositions up to N (included), and proofs up to N-1 (included; if N=1, then there is no proof to include in the context). 
(2) Note when proof N uses a resource has not appeared anywhere in the context (i.e. in the definitions, postulates, common notions, propositions up to N included, and proofs up to N-1 included).

PREPARATION: review 
    - main.tex (for the definition of typical activation potential), 
    - analyses.ipynb (for strategies to compare proofs with their context of previous material), and 
    - typical.ipynb (for an algorithm implementing typical activation potential)
    - ./modules/queries.py (for SPARQL queries used to work with ontological resources)

NOTES:
    - cache results of SPARQL queries for faster re-runs (QueryRunner)

INPUTS: PROOF_N, DELTA, HISTORY_WEIGHTS, TYPE_SELECTION

HOW TO CREATE THE CONTEXT OF PROOF N:
- directly used resources: use queries.direct_definitions(), queries.direct_postulates(), queries.direct_common_notions, and queries.direct_template_propositions_proofs() for propositions up to N (included) and proofs up to N-1 (included) [for propositions and proofs, one need to state the IRIs as VALUES in the SPARQL queries]
- hierarchically used resources: use queries.hierarchical_definitions(), queries.hierarchical_postulates(), queries.hierarchical_common_notions(), and hierarchical_template_propositions_proofs for propositions up to N (included) and proofs up to N-1 (included) [for propositions and proofs, one need to state the IRIs as VALUES in the SPARQL queries]
- mereologically used resources: use queries.mereological_definitions(), queries.mereological_postulates(), queries.mereological_common_notions, queries.mereological_template_propositions_proofs() for propositions up to N (included) and proofs up to N-1 (included) [for propositions and proofs, one need to state the IRIs as VALUES in the SPARQL queries]
- hebbian co-occurrence: use queries.hebb_definitions(), queries.hebb_postulates(), queries.hebb_common_notions, and queries.hebb_template_propositions_proofs() for propositions up to N (included) and proofs up to N-1 (included) [for propositions and proofs, one need to state the IRIs as VALUES in the SPARQL queries].

EDGE CASE:
- N = 1: the context includes only definitions, postulates, common notions, and proposition 1.

HOW TO FIND RESOURCES IN PROOF N:
- if TYPE_SELECTION = False: use queries.direct_template_propositions_proofs() for proof N (the list of values for the query should contain only the IRI of proof N)
- if TYPE_SELECTION = True: use queries.direct_template_last_item_types() for proof N (the list of values for the query should contain only the IRI of proof N)

OUTPUT: 
    - csv with columns "proof", "resource_used_in_proof", "number_of_resources_used_in_proof", "phi_h", "phi_tc", "phi_t", "new_resources", "number_of_new_resources"
    - include only resources used in proof N (for each N) in the column "resources_used_in_proof"
    - output path: put the csv file in the "./output" folder
    - output naming convention: typical_weights-<history_weights>_type-<true_or_false>_<timestamp>.csv
    - "new_resources" contains a list of resources that are new in proof N (never used before)
    - "new_resources", "number_of_new_resources" are per-proof data; however I want to include them in a single csv; therefore, it is ok to repeat these data on several rows for the same proof

In [ ]:
from __future__ import annotations

import datetime as dt
from pathlib import Path

import pandas as pd

from modules import rdf_utils, file_utils
from modules.calculate_activation_potential import history as history_potential
from modules.calculate_activation_potential import hebb as hebb_potential
from modules.query_runner import QueryRunner